## Preparation

In [1]:
%%capture
!conda install -c pykaldi pykaldi -y

In [2]:
%cd /tmp

/tmp


In [3]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

Cloning into 'fairseq'...
remote: Enumerating objects: 28209, done.
remote: Counting objects: 100% (621/621), done.
remote: Compressing objects: 100% (356/356), done.
remote: Total 28209 (delta 329), reused 494 (delta 250), pack-reused 27588
Receiving objects: 100% (28209/28209), 11.88 MiB | 18.29 MiB/s, done.
Resolving deltas: 100% (21148/21148), done.


In [4]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 17.97 MiB/s, done.
Resolving deltas: 100% (7956/7956), done.


In [5]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [6]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [7]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [8]:
%cd /tmp/fairseq/

/tmp/fairseq


In [9]:
!pip install .

Processing /tmp/fairseq
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / done
    Preparing wheel metadata ... - \ done
     |████████████████████████████████| 123 kB 8.0 MB/s 
     |████████████████████████████████| 54 kB 1.8 MB/s 
     |████████████████████████████████| 112 kB 63.3 MB/s 
  Created wheel for fairseq: filename=fairseq-1.0.0a0+5744f04-cp37-cp37m-linux_x86_64.whl size=2041102 sha256=378264b0c1918c596b1764e6191a3cb7f95512b107a3f55a7258298aa9e60ebd
  Stored in directory: /tmp/pip-ephem-wheel-cache-jfboxydl/wheels/e3/52/72/c9070e4abac84e3448568d15e13e9e42bb689c08614271d8f3
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=eb1e517c04b618f170907f3b41a92082c690ead00bb12f812f361db982f06075
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Suc

In [10]:
%cd /tmp/fairseq/examples/wav2vec/unsupervised

/tmp/fairseq/examples/wav2vec/unsupervised


In [11]:
os.environ['HYDRA_FULL_ERROR'] = '1'

## GAN

In [12]:
!ls /kaggle/input/wav2vec-u-cv-swedish-audio/

CLUS128		    precompute_pca512_cls128_mean	  train.npy
__notebook__.ipynb  precompute_pca512_cls128_mean_pooled  train.phn
__output__.json     test.lengths			  train.tsv
__results__.html    test.ltr				  train.wrd
custom.css	    test.npy				  valid.lengths
dict.test	    test.phn				  valid.ltr
dict.train	    test.tsv				  valid.npy
dict.valid	    test.wrd				  valid.phn
pca		    train.lengths			  valid.tsv
precompute_pca512   train.ltr				  valid.wrd


In [13]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir model/wav2vecu/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

Writing rungan.sh


In [14]:
!bash rungan.sh

Traceback (most recent call last):
  File "/opt/conda/bin/fairseq-hydra-train", line 5, in <module>
    from fairseq_cli.hydra_train import cli_main
  File "/opt/conda/lib/python3.7/site-packages/fairseq_cli/hydra_train.py", line 10, in <module>
    from fairseq.dataclass.initialize import add_defaults, hydra_init
  File "/opt/conda/lib/python3.7/site-packages/fairseq/__init__.py", line 32, in <module>
    import fairseq.criterions  # noqa
  File "/opt/conda/lib/python3.7/site-packages/fairseq/criterions/__init__.py", line 36, in <module>
    importlib.import_module("fairseq.criterions." + file_name)
  File "/opt/conda/lib/python3.7/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/opt/conda/lib/python3.7/site-packages/fairseq/criterions/ctc.py", line 19, in <module>
    from fairseq.tasks import FairseqTask
  File "/opt/conda/lib/python3.7/site-packages/fairseq/tasks/__init__.py", line 116, in <module>
    modul